In [1]:
import os
import pickle
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.utils.data as data1
from torch.utils import data
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

In [2]:
# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, frame_length=10, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path
        self.frames = frame_length #For our case since we are computing 10 frames always

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        file_name = ""
        for i in os.listdir(data_path):
            file_name = i
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)

        return X, file_name

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
              
        # Load data
        X, file_name = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = np.ones(self.frames)
        if 'real' in data_path:
            y = np.zeros(self.frames)
        #print("Folder is {}".format(data_path))
        #print(X.shape)
        return X, torch.from_numpy(y).type(torch.LongTensor)

In [3]:
TRANSFORM_IMG_TRAIN = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(256),
    transforms.RandomResizedCrop(256)
    
    ])

In [4]:
TRANSFORM_IMG_VAL = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(256)
])

In [5]:
train_path = '/home/chinmay/datatset/deepfake_split/train'
train_data = Dataset_CRNN(train_path, transform=TRANSFORM_IMG_TRAIN, frame_length=30 )
# for step, (x, y) in enumerate(data):
#     print(x.shape)
val_path = '/home/chinmay/datatset/deepfake_split/val'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG_VAL, frame_length=30)

In [6]:
epochs = 80
batch_size = 4
learning_rate = 1e-4
log_interval = 10
img_x, img_y = 96,96#128,128#256, 256  # resize video 2d frame size

In [7]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU
print("Is use_cuda", use_cuda)
# Now load the dataset
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
# Load the dataset

train_loader = data1.DataLoader(train_data, **params)
valid_loader = data1.DataLoader(val_data, **params)

Is use_cuda True


In [8]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    confidence_scores = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
        # Let us compute the confidence of the scores
        # since frames are independent, our confidence is purely based on the number
        # of frames our model thinks is belonging to a specific category
        # the confidence of individual frame prediction is not taken into consideration
        # and this portion is debatable....
        frame_set_pred = np.sort(counts)[-1]
        confidence = frame_set_pred/sum(counts)
        confidence_scores.append(confidence)
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor), torch.from_numpy(np.asarray(confidence_scores)).type(torch.FloatTensor)

In [9]:
# Model paths
writer_train = SummaryWriter('/home/chinmay/training-results/conv3D_refined_2/train')
writer_test = SummaryWriter('/home/chinmay/training-results/conv3D_refined_2/test')
save_model_path = "/home/chinmay/model_weights/conv3D_refined/"



# Select which frame to begin & end in videos
begin_frame, end_frame, skip_frame = 1, 10, 1

def train(log_interval, model, device, train_loader, optimizer, epoch):
    # set model as training mode
    model.train()

    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        #print("Shape of X is {}".format(X.shape))
        # distribute data to device
        #X, y = X.to(device), y.to(device)
        X, y = X.cuda(), y.cuda()
        #print("The label is ",y)
        N_count += X.size(0)
        #print("The size is ",X.size())
        optimizer.zero_grad()
        output = model(X)  # output size = (batch, number of classes)
        y, _ = find_median(y) #This is necessary as now only single label output for entire frame
        y = y.to(device)
        #print(y)
        #print(y.shape)
        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()
        
                 
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
         
    return np.mean(losses), np.mean(scores)

In [10]:
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    all_y = []
    all_y_pred = []
    test_loss = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            #X, y = X.to(device), y.to(device)
            X, y = X.cuda(), y.cuda()
            output = model(X)

            y, _ = find_median(y) #This is necessary as now only single label output for entire frame
            y = y.to(device)
            loss = F.cross_entropy(output, y)
            test_loss.append(loss.item())                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability
            
            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss = np.mean(test_loss)

    # to compute accuracy
#     all_y = torch.stack(all_y, dim=0)
#     all_y_pred = torch.stack(all_y_pred, dim=0)
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, 'cnn3d{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'cnn3d_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))


    return test_loss, test_score

In [ ]:
# create model
#cnn3d = CNN3D(t_dim=10, img_x=img_x, img_y=img_y,
#              drop_p=dropout, fc_hidden1=fc_hidden1,  fc_hidden2=fc_hidden2, num_classes=2)

from Res3D import C3D
cnn3d = C3D(img_dim=256, frames=30, dropout=0.25)
# Comment this part...
model_path = "/home/chinmay/model_weights/conv3D_refined/"
weight_location = os.path.join(model_path,"cnn3d4.pth")
cnn3d.load_state_dict(torch.load(weight_location))
#

cnn3d.cuda()
# Parallelize model to multiple GPUs
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    cnn3d = nn.DataParallel(cnn3d)

optimizer = torch.optim.Adam(cnn3d.parameters(), lr=learning_rate)   # optimize all cnn parameters


# record training process
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []


# start training
for epoch in range(epochs):
    # train, test model
    train_losses, train_scores = train(log_interval, cnn3d, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(cnn3d, device, optimizer, valid_loader)

    # save all train test results
    # save results
    writer_train.add_scalar('loss',train_losses,epoch+1)
    writer_train.add_scalar('score',train_scores,epoch+1)
    writer_test.add_scalar('loss',epoch_test_loss,epoch+1)
    writer_test.add_scalar('score',epoch_test_score,epoch+1)
    
    
    torch.cuda.empty_cache()


Train Epoch: 1 [40/416 (10%)]	Loss: 0.271120, Accu: 100.00%
Train Epoch: 1 [80/416 (19%)]	Loss: 0.452481, Accu: 75.00%
Train Epoch: 1 [120/416 (29%)]	Loss: 0.490022, Accu: 75.00%
Train Epoch: 1 [160/416 (38%)]	Loss: 1.449256, Accu: 50.00%
Train Epoch: 1 [200/416 (48%)]	Loss: 0.882282, Accu: 50.00%
Train Epoch: 1 [240/416 (58%)]	Loss: 0.937742, Accu: 50.00%
Train Epoch: 1 [280/416 (67%)]	Loss: 0.897829, Accu: 50.00%
Train Epoch: 1 [320/416 (77%)]	Loss: 1.139328, Accu: 50.00%
Train Epoch: 1 [360/416 (87%)]	Loss: 1.362426, Accu: 50.00%
Train Epoch: 1 [400/416 (96%)]	Loss: 0.162122, Accu: 100.00%

Test set (235 samples): Average loss: 0.7853, Accuracy: 60.00%

Epoch 1 model saved!
Train Epoch: 2 [40/416 (10%)]	Loss: 0.294280, Accu: 100.00%
Train Epoch: 2 [80/416 (19%)]	Loss: 0.530103, Accu: 75.00%
Train Epoch: 2 [120/416 (29%)]	Loss: 0.535219, Accu: 75.00%
Train Epoch: 2 [160/416 (38%)]	Loss: 0.931887, Accu: 50.00%
Train Epoch: 2 [200/416 (48%)]	Loss: 0.248920, Accu: 100.00%
Train Epoch: 2


Test set (235 samples): Average loss: 0.4314, Accuracy: 79.57%

Epoch 12 model saved!
Train Epoch: 13 [40/416 (10%)]	Loss: 0.203090, Accu: 100.00%
Train Epoch: 13 [80/416 (19%)]	Loss: 0.267791, Accu: 75.00%
Train Epoch: 13 [120/416 (29%)]	Loss: 0.721068, Accu: 75.00%
Train Epoch: 13 [160/416 (38%)]	Loss: 0.342566, Accu: 100.00%
Train Epoch: 13 [200/416 (48%)]	Loss: 0.215494, Accu: 100.00%
Train Epoch: 13 [240/416 (58%)]	Loss: 0.417917, Accu: 75.00%
Train Epoch: 13 [280/416 (67%)]	Loss: 1.194314, Accu: 50.00%
Train Epoch: 13 [320/416 (77%)]	Loss: 0.687885, Accu: 50.00%
Train Epoch: 13 [360/416 (87%)]	Loss: 0.173987, Accu: 100.00%
Train Epoch: 13 [400/416 (96%)]	Loss: 0.167501, Accu: 100.00%

Test set (235 samples): Average loss: 0.5693, Accuracy: 78.30%

Epoch 13 model saved!
Train Epoch: 14 [40/416 (10%)]	Loss: 0.369195, Accu: 75.00%
Train Epoch: 14 [80/416 (19%)]	Loss: 0.127787, Accu: 100.00%
Train Epoch: 14 [120/416 (29%)]	Loss: 0.468968, Accu: 75.00%
Train Epoch: 14 [160/416 (38%)]